In [1]:
# Importing libraries


import pandas as pd
import numpy as np
%matplotlib inline

from sklearn.metrics import mean_squared_log_error
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor


import pandas_profiling
import keras
from keras.models import Sequential
from keras.layers import Dense

from sklearn import preprocessing
le = preprocessing.LabelEncoder()

Using TensorFlow backend.


In [2]:
# Reading data to Dataframes
  
calendar = pd.read_csv('../M5 Forecasting/calendar.csv')
sales_train_validation = pd.read_csv('../M5 Forecasting/sales_train_validation.csv')
sell_prices = pd.read_csv('../M5 Forecasting/sell_prices.csv')

In [3]:
sales_train_validation.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


In [4]:
#teste = sales_train_validation.query('item_id == "HOBBIES_1_001"')
teste = sales_train_validation.query('state_id == "CA"')
#teste = sales_train_validation
teste.describe()

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
count,12196.000000,12196.000000,12196.000000,12196.000000,12196.000000,12196.000000,12196.000000,12196.000000,12196.000000,12196.000000,...,12196.000000,12196.000000,12196.000000,12196.000000,12196.000000,12196.000000,12196.000000,12196.000000,12196.000000,12196.000000
mean,1.163906,1.131929,0.828796,0.905789,0.813791,0.928337,1.004510,1.361922,1.204985,0.969334,...,1.332814,1.686127,1.888488,1.398163,1.294195,1.242047,1.187930,1.401689,1.790259,1.901197
std,5.347562,6.045263,3.707162,4.882840,4.222357,4.875753,5.639758,6.791817,6.235198,5.084530,...,3.234806,4.025610,4.687866,3.357752,3.123486,2.912577,2.917047,3.386963,4.016504,4.084711
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,...,2.000000,2.000000,2.000000,2.000000,1.000000,1.000000,1.000000,2.000000,2.000000,2.000000
max,360.000000,436.000000,207.000000,323.000000,296.000000,314.000000,316.000000,370.000000,385.000000,353.000000,...,129.000000,160.000000,204.000000,86.000000,100.000000,88.000000,77.000000,141.000000,139.000000,130.000000


In [5]:
#pivotando as colunas

melt = teste.melt(id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name='Days', value_name='Sales')


In [6]:
melt = melt.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'] , axis = 1)




In [7]:
df = melt

In [8]:
#label encoder
def generate_labelencoder(atts):
  for attr in atts:
    df[attr] = le.fit_transform(df[attr])
  return df

df = generate_labelencoder(['id','Days'])  

In [9]:
#adicionando dias anteriores na tupla

melt_temp = melt.copy()
melt_temp['Sales_1'] = melt_temp.groupby(['id'])['Sales'].shift()
melt_temp['Sales_2'] = melt_temp.groupby(['id'])['Sales'].shift(2)
melt_temp['Sales_3'] = melt_temp.groupby(['id'])['Sales'].shift(3)
melt_temp['Sales_4'] = melt_temp.groupby(['id'])['Sales'].shift(4)
melt_temp['Sales_5'] = melt_temp.groupby(['id'])['Sales'].shift(5)
melt_temp['Sales_6'] = melt_temp.groupby(['id'])['Sales'].shift(6)
melt_temp['Sales_7'] = melt_temp.groupby(['id'])['Sales'].shift(7)
melt_temp = melt_temp.dropna()
melt_temp.head(5)

,id,Days,Sales,Sales_1,Sales_2,Sales_3,Sales_4,Sales_5,Sales_6,Sales_7
85372,5748,1691,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
85373,5752,1691,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
85374,5756,1691,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
85375,5760,1691,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
85376,5764,1691,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
#tentando diminuir o consumo de memoria
melt_temp = melt_temp.astype('int32')
melt_temp = melt_temp.query('Days > 1682')

In [11]:
melt_temp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2805080 entries, 85372 to 12183803
Data columns (total 10 columns):
 #   Column   Dtype
---  ------   -----
 0   id       int32
 1   Days     int32
 2   Sales    int32
 3   Sales_1  int32
 4   Sales_2  int32
 5   Sales_3  int32
 6   Sales_4  int32
 7   Sales_5  int32
 8   Sales_6  int32
 9   Sales_7  int32
dtypes: int32(10)
memory usage: 128.4 MB


In [12]:
def rmsle(ytrue, ypred):
    return np.sqrt(mean_squared_log_error(ytrue, ypred))

In [13]:
#criando o modelo
melt = melt_temp

mean_error = []
for days in range(1884,1912):
    train = melt[melt['Days'] < days]
    val = melt[melt['Days'] == days]
    
    x_train, x_test = train.drop(['Sales'], axis=1), val.drop(['Sales'], axis=1)
    y_train, y_test = train['Sales'].values, val['Sales'].values
    
    mdl = RandomForestRegressor(n_estimators=100, n_jobs=-1, random_state=0 , verbose= 2)
    mdl.fit(x_train, y_train)
    
    p = mdl.predict(x_test)
    
    error = rmsle(y_test, p)
    print('Day %d - Error %.5f' % (days, error))
    mean_error.append(error)
print('Mean Error = %.5f' % np.mean(mean_error))



n_jobs=4)]: Done 100 out of 100 | elapsed:   10.7s finished
Day 1904 - Error 0.56617
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
building tree 1 of 100building tree 2 of 100building tree 3 of 100
building tree 4 of 100


building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100building tree 14 of 100

building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
bui